# ETL Extract

1.   Extract domains/url classification
2.   Get the gender from the first name of the employees
3.   Confirmed insider cases



The classification of URL was obtained from the web link
[link text](http://www.shallalist.de/)
It contains directories named after each classification, each directory has two files domains and url e.g. advertisement

adv

  -> domains

  -> urls

In [0]:
# execute to install not base libraries
#pip install genderize


In [0]:
import os    #library to join path names
import pandas as pd #data manipulation library using data frames
import glob #library to walk directory paths
from genderize import Genderize #library to connect to Genderize api
import json #library to call json files (a.k.a by UP DS master students jotason )
from dateutil.relativedelta import relativedelta # library to add/substract dates
from datetime import datetime #library for dates manipulation


In [0]:
#Global path variables
bl_path='./drive/My Drive/DatosInsider/BL/' # path with all directories for URL classification
emp_path = './drive/My Drive/DatosInsider/r4.2/LDAP/' #path with employee information
dwh_path ='./drive/My Drive/DatosInsider/DWH_tables/' #final repository path
cnf_ins_path ='./drive/My Drive/DatosInsider/answers/' #path with confirmed Insider cases

In [7]:
domain_catalogue= pd.DataFrame(columns=['type','domain_url','name'])
listOfFiles = list()
for (dirpath, bl_path, filenames) in os.walk(bl_path):
    listOfFiles += [os.path.join(dirpath, file) for file in filenames]
        
for file in listOfFiles:
  typef= file.split('/')[-2]
  domain_url= file.split('/')[-1]
  if domain_url in ['domains','urls']:
    try:
      aux_df=pd.read_csv(file, header=None,delimiter='\n',encoding='latin')
      aux_df.columns=['name']
      aux_df['type']=typef
      aux_df['domain_url']=domain_url
      aux_df= aux_df[['type','domain_url','name']]
      domain_catalogue=domain_catalogue.append(aux_df)
    except:
      print(file)

./drive/My Drive/DatosInsider/BL/dynamic/urls


In [0]:
#Finally save results into the work repository
domain_catalogue.to_csv(os.path.join(dwh_path, 'BL_domains.csv') , index=False)
del(domain_catalogue)

LDAP directory from the CERT dataset contains information from all employees. Employees first name will be extracted and then using the Genderize api gender will be infered

In [0]:
#Get data from all the files in the Path LDAP that includes
# employee information
all_employee_files = glob.glob(emp_path + "*.csv")
list_df = []
for filename in all_employee_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    list_df.append(df)

employees = pd.concat(list_df, axis=0, ignore_index=True)
#keep only name and ID and get unique employees 
employees= employees[['user_id', 'employee_name']]
employees.drop_duplicates( "user_id" ,inplace=True)

In [0]:
#Using an API get the gender of the employee based on the first name
# Details on https://genderize.io/  https://github.com/SteelPangolin/genderize

employees['gender']=""
employees['gender_probability']=""
for idx in range(len(employees)):
  first_name =employees.loc[idx, 'employee_name' ].split(" ")[0]  
  genero=Genderize().get([first_name] )
  employees['gender'].iloc[idx]= genero[0]['gender']
  employees['gender_probability'].iloc[idx]=genero[0]['probability']

In [0]:
#Finally save results into the work repository
employees.to_csv(os.path.join(dwh_path, 'employee_data.csv') )
del(employees)

From the CERT data set

This is a dataset of all true positives where the Scenario column descrives the following.

User who did not previously use removable drives or work after hours begins logging in after hours, using a removable drive, and uploading data to wikileaks.org. Leaves the organization shortly thereafter.

User begins surfing job websites and soliciting employment from a competitor. Before leaving the company, they use a thumb drive (at markedly higher rates than their previous activity) to steal data.

System administrator becomes disgruntled. Downloads a keylogger and uses a thumb drive to transfer it to his supervisor's machine. The next day, he uses the collected keylogs to log in as his supervisor and send out an alarming mass email, causing panic in the organization. He leaves the organization immediately.

Thus, 1 can be coded as Leaker, 2 as Thief and 3 as Saboteur

In [0]:
insider= pd.read_csv(os.path.join(cnf_ins_path,'insiders.csv')) 
# All data points will be considred montly. Thus, data will be transformed.
#only get dataset of interest
insider=insider[insider.dataset==4.2]
#transform to datetime format
insider['start']= pd.to_datetime( insider['start'],format='%m/%d/%Y %H:%M')
insider['end']= pd.to_datetime( insider['end'],format='%m/%d/%Y %H:%M')
#get the number of months that an insider was active i.e. committed a crime
insider['active_periods']= insider.apply( lambda x: (x.end.year - x.start.year) * 12 + (x.end.month - x.start.month),axis=1)

In [14]:

#for each period that the insider was active create a new row in a dataframe
insider_cols=['user','period']
insider_events=pd.DataFrame(columns=insider_cols)
for _,ins in insider.iterrows():
  for idx in range(0,ins.active_periods+1):
    new_date=ins.start +relativedelta(months=+ idx)
    period= new_date.strftime('%Y%m')
    fields=[ins.user, period]
    insider_events=insider_events.append(pd.Series(fields, index=insider_cols),ignore_index=True)   
insider_events.shape


(128, 2)

In [0]:
insider_events.to_csv("./drive/My Drive/DatosInsider/DWH_tables/insider_confirmed_cases.csv")
del(insider_events)